## Recommender System for Movies Using Word2Vec and TFIDF
* Used the TMDB5000 Dataset to create a content-based recommender system
* Took inspiration from this [Notebook](https://www.kaggle.com/code/ibtesama/getting-started-with-a-movie-recommendation-system) by [@ibtesama](https://www.kaggle.com/ibtesama)

### Basic Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

import warnings
warnings.filterwarnings('ignore')

### Processing the Dataset

In [2]:
movie_path = '/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv'
credit_path = '/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv'

In [3]:
movie_df = pd.read_csv(movie_path)
credit_df = pd.read_csv(credit_path)

In [4]:
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [5]:
credit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4803 non-null   int64 
 1   title     4803 non-null   object
 2   cast      4803 non-null   object
 3   crew      4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


In [6]:
overall_df = pd.merge(movie_df, credit_df)

In [7]:
overall_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [8]:
overall_df

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4804,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]",...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,9367,"[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de..."
4805,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,72766,"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"":

* Here are some observations:
    * `id` and `movie_id` are the same
    *  features like `budget`, `homepage`, `original_title`, `production_companies`, `production_countries`, `revenue`, `spoken_languages`, `status`, `tagline`, `vote_average`, `vote_count` are kind of redundant for the recommendation system that we are making here so, we're gonna drop them
    

In [9]:
overall_df = overall_df.drop(['id', 'budget', 'homepage', 'original_title', 'production_companies', 'production_countries', 'revenue', 'spoken_languages', 'status', 'tagline', 'vote_average', 'vote_count'], axis = 1)
overall_df

,genres,keywords,original_language,overview,popularity,release_date,runtime,title,movie_id,cast,crew
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,"In the 22nd century, a paraplegic Marine is di...",150.437577,2009-12-10,162.0,Avatar,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,"Captain Barbossa, long believed to be dead, ha...",139.082615,2007-05-19,169.0,Pirates of the Caribbean: At World's End,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,A cryptic message from Bond’s past sends him o...,107.376788,2015-10-26,148.0,Spectre,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,Following the death of District Attorney Harve...,112.312950,2012-07-16,165.0,The Dark Knight Rises,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,"John Carter is a war-weary, former military ca...",43.926995,2012-03-07,132.0,John Carter,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
...,...,...,...,...,...,...,...,...,...,...,...
4804,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi just wants to play his guitar and ...,14.269792,1992-09-04,81.0,El Mariachi,9367,"[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de..."
4805,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",[],en,A newlywed couple's honeymoon is upended by th...,0.642552,2011-12-26,85.0,Newlyweds,72766,"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de..."
4806,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,2013-10-13,120.0,"Signed, Sealed, Delivered",231617,"[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de..."
4807,[],[],en,When ambitious New York attorney Sam is sent t...,0.857008,2012-05-03,98.0,Shanghai Calling,126186,"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de..."


### Data Cleaning and Processing
#### Genres, Cast, Keywords

In [10]:
overall_df['genres'][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

It's a list of JSON objects which have genres in front of "name". Similar for keywords and cast

In [11]:
def extract_names(cell_value):
    try:
        data_list = json.loads(cell_value)
        if(all):
            return [entry["name"] for entry in data_list]
        else:
            return [entry["name"] for entry in data_list[:top]]
    except json.JSONDecodeError:
        return []

In [12]:
overall_df['genres'] = overall_df['genres'].apply(extract_names)

In [13]:
overall_df['keywords'] = overall_df['keywords'].apply(extract_names)

In [14]:
overall_df['cast'] = overall_df['cast'].apply(extract_names)

In [15]:
overall_df

,genres,keywords,original_language,overview,popularity,release_date,runtime,title,movie_id,cast,crew
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,"In the 22nd century, a paraplegic Marine is di...",150.437577,2009-12-10,162.0,Avatar,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...",en,"Captain Barbossa, long believed to be dead, ha...",139.082615,2007-05-19,169.0,Pirates of the Caribbean: At World's End,285,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",en,A cryptic message from Bond’s past sends him o...,107.376788,2015-10-26,148.0,Spectre,206647,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",en,Following the death of District Attorney Harve...,112.312950,2012-07-16,165.0,The Dark Knight Rises,49026,"[Christian Bale, Michael Caine, Gary Oldman, A...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...",en,"John Carter is a war-weary, former military ca...",43.926995,2012-03-07,132.0,John Carter,49529,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
...,...,...,...,...,...,...,...,...,...,...,...
4804,"[Action, Crime, Thriller]","[united states–mexico barrier, legs, arms, pap...",es,El Mariachi just wants to play his guitar and ...,14.269792,1992-09-04,81.0,El Mariachi,9367,"[Carlos Gallardo, Jaime de Hoyos, Peter Marqua...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de..."
4805,"[Comedy, Romance]",[],en,A newlywed couple's honeymoon is upended by th...,0.642552,2011-12-26,85.0,Newlyweds,72766,"[Edward Burns, Kerry Bishé, Marsha Dietlein, C...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de..."
4806,"[Comedy, Drama, Romance, TV Movie]","[date, love at first sight, narration, investi...",en,"""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,2013-10-13,120.0,"Signed, Sealed, Delivered",231617,"[Eric Mabius, Kristin Booth, Crystal Lowe, Geo...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de..."
4807,[],[],en,When ambitious New York attorney Sam is sent t...,0.857008,2012-05-03,98.0,Shanghai Calling,126186,"[Daniel Henney, Eliza Coupe, Bill Paxton, Alan...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de..."


#### Crew
* We'll just focus on the director over here

In [16]:
def extract_director_name(crew_str):
    try:
        crew_list = json.loads(crew_str)
        director_name = next((crew_dict.get('name') for crew_dict in crew_list if crew_dict.get('job') == 'Director'), None)
        return director_name
    except json.JSONDecodeError:
        return None

In [17]:
overall_df['director'] = overall_df['crew'].apply(extract_director_name)

In [18]:
overall_df.drop(['crew'], axis = 1, inplace = True)

In [19]:
overall_df

,genres,keywords,original_language,overview,popularity,release_date,runtime,title,movie_id,cast,director
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,"In the 22nd century, a paraplegic Marine is di...",150.437577,2009-12-10,162.0,Avatar,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",James Cameron
1,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...",en,"Captain Barbossa, long believed to be dead, ha...",139.082615,2007-05-19,169.0,Pirates of the Caribbean: At World's End,285,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...",Gore Verbinski
2,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",en,A cryptic message from Bond’s past sends him o...,107.376788,2015-10-26,148.0,Spectre,206647,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",Sam Mendes
3,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",en,Following the death of District Attorney Harve...,112.312950,2012-07-16,165.0,The Dark Knight Rises,49026,"[Christian Bale, Michael Caine, Gary Oldman, A...",Christopher Nolan
4,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...",en,"John Carter is a war-weary, former military ca...",43.926995,2012-03-07,132.0,John Carter,49529,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...
4804,"[Action, Crime, Thriller]","[united states–mexico barrier, legs, arms, pap...",es,El Mariachi just wants to play his guitar and ...,14.269792,1992-09-04,81.0,El Mariachi,9367,"[Carlos Gallardo, Jaime de Hoyos, Peter Marqua...",Robert Rodriguez
4805,"[Comedy, Romance]",[],en,A newlywed couple's honeymoon is upended by th...,0.642552,2011-12-26,85.0,Newlyweds,72766,"[Edward Burns, Kerry Bishé, Marsha Dietlein, C...",Edward Burns
4806,"[Comedy, Drama, Romance, TV Movie]","[date, love at first sight, narration, investi...",en,"""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,2013-10-13,120.0,"Signed, Sealed, Delivered",231617,"[Eric Mabius, Kristin Booth, Crystal Lowe, Geo...",Scott Smith
4807,[],[],en,When ambitious New York attorney Sam is sent t...,0.857008,2012-05-03,98.0,Shanghai Calling,126186,"[Daniel Henney, Eliza Coupe, Bill Paxton, Alan...",Daniel Hsia


#### Releasing date
* We can extract the year from this feature

In [20]:
def extract_years(date):
    x = str(date)
    return x[0:4]

In [21]:
overall_df['release_date'].apply(extract_years)

0       2009
1       2007
2       2015
3       2012
4       2012
        ... 
4804    1992
4805    2011
4806    2013
4807    2012
4808    2005
Name: release_date, Length: 4809, dtype: object

In [22]:
overall_df

,genres,keywords,original_language,overview,popularity,release_date,runtime,title,movie_id,cast,director
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,"In the 22nd century, a paraplegic Marine is di...",150.437577,2009-12-10,162.0,Avatar,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",James Cameron
1,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...",en,"Captain Barbossa, long believed to be dead, ha...",139.082615,2007-05-19,169.0,Pirates of the Caribbean: At World's End,285,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...",Gore Verbinski
2,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",en,A cryptic message from Bond’s past sends him o...,107.376788,2015-10-26,148.0,Spectre,206647,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",Sam Mendes
3,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",en,Following the death of District Attorney Harve...,112.312950,2012-07-16,165.0,The Dark Knight Rises,49026,"[Christian Bale, Michael Caine, Gary Oldman, A...",Christopher Nolan
4,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...",en,"John Carter is a war-weary, former military ca...",43.926995,2012-03-07,132.0,John Carter,49529,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...
4804,"[Action, Crime, Thriller]","[united states–mexico barrier, legs, arms, pap...",es,El Mariachi just wants to play his guitar and ...,14.269792,1992-09-04,81.0,El Mariachi,9367,"[Carlos Gallardo, Jaime de Hoyos, Peter Marqua...",Robert Rodriguez
4805,"[Comedy, Romance]",[],en,A newlywed couple's honeymoon is upended by th...,0.642552,2011-12-26,85.0,Newlyweds,72766,"[Edward Burns, Kerry Bishé, Marsha Dietlein, C...",Edward Burns
4806,"[Comedy, Drama, Romance, TV Movie]","[date, love at first sight, narration, investi...",en,"""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,2013-10-13,120.0,"Signed, Sealed, Delivered",231617,"[Eric Mabius, Kristin Booth, Crystal Lowe, Geo...",Scott Smith
4807,[],[],en,When ambitious New York attorney Sam is sent t...,0.857008,2012-05-03,98.0,Shanghai Calling,126186,"[Daniel Henney, Eliza Coupe, Bill Paxton, Alan...",Daniel Hsia


In [23]:
overall_df.drop(['release_date'], axis = 1, inplace = True)

In [24]:
overall_df

,genres,keywords,original_language,overview,popularity,runtime,title,movie_id,cast,director
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,"In the 22nd century, a paraplegic Marine is di...",150.437577,162.0,Avatar,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",James Cameron
1,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...",en,"Captain Barbossa, long believed to be dead, ha...",139.082615,169.0,Pirates of the Caribbean: At World's End,285,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...",Gore Verbinski
2,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",en,A cryptic message from Bond’s past sends him o...,107.376788,148.0,Spectre,206647,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",Sam Mendes
3,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",en,Following the death of District Attorney Harve...,112.312950,165.0,The Dark Knight Rises,49026,"[Christian Bale, Michael Caine, Gary Oldman, A...",Christopher Nolan
4,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...",en,"John Carter is a war-weary, former military ca...",43.926995,132.0,John Carter,49529,"[Taylor Kitsch, Lynn Collins, Samantha Morton,...",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...
4804,"[Action, Crime, Thriller]","[united states–mexico barrier, legs, arms, pap...",es,El Mariachi just wants to play his guitar and ...,14.269792,81.0,El Mariachi,9367,"[Carlos Gallardo, Jaime de Hoyos, Peter Marqua...",Robert Rodriguez
4805,"[Comedy, Romance]",[],en,A newlywed couple's honeymoon is upended by th...,0.642552,85.0,Newlyweds,72766,"[Edward Burns, Kerry Bishé, Marsha Dietlein, C...",Edward Burns
4806,"[Comedy, Drama, Romance, TV Movie]","[date, love at first sight, narration, investi...",en,"""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,120.0,"Signed, Sealed, Delivered",231617,"[Eric Mabius, Kristin Booth, Crystal Lowe, Geo...",Scott Smith
4807,[],[],en,When ambitious New York attorney Sam is sent t...,0.857008,98.0,Shanghai Calling,126186,"[Daniel Henney, Eliza Coupe, Bill Paxton, Alan...",Daniel Hsia


* Getting just the top cast members of the list

In [25]:
def get_top_cast(list):
    return list[:3]

In [26]:
overall_df['cast'] = overall_df['cast'].apply(get_top_cast)

In [27]:
overall_df

,genres,keywords,original_language,overview,popularity,runtime,title,movie_id,cast,director
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,"In the 22nd century, a paraplegic Marine is di...",150.437577,162.0,Avatar,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron
1,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...",en,"Captain Barbossa, long believed to be dead, ha...",139.082615,169.0,Pirates of the Caribbean: At World's End,285,"[Johnny Depp, Orlando Bloom, Keira Knightley]",Gore Verbinski
2,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",en,A cryptic message from Bond’s past sends him o...,107.376788,148.0,Spectre,206647,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",Sam Mendes
3,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",en,Following the death of District Attorney Harve...,112.312950,165.0,The Dark Knight Rises,49026,"[Christian Bale, Michael Caine, Gary Oldman]",Christopher Nolan
4,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...",en,"John Carter is a war-weary, former military ca...",43.926995,132.0,John Carter,49529,"[Taylor Kitsch, Lynn Collins, Samantha Morton]",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...
4804,"[Action, Crime, Thriller]","[united states–mexico barrier, legs, arms, pap...",es,El Mariachi just wants to play his guitar and ...,14.269792,81.0,El Mariachi,9367,"[Carlos Gallardo, Jaime de Hoyos, Peter Marqua...",Robert Rodriguez
4805,"[Comedy, Romance]",[],en,A newlywed couple's honeymoon is upended by th...,0.642552,85.0,Newlyweds,72766,"[Edward Burns, Kerry Bishé, Marsha Dietlein]",Edward Burns
4806,"[Comedy, Drama, Romance, TV Movie]","[date, love at first sight, narration, investi...",en,"""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,120.0,"Signed, Sealed, Delivered",231617,"[Eric Mabius, Kristin Booth, Crystal Lowe]",Scott Smith
4807,[],[],en,When ambitious New York attorney Sam is sent t...,0.857008,98.0,Shanghai Calling,126186,"[Daniel Henney, Eliza Coupe, Bill Paxton]",Daniel Hsia


### Text Processing
We'll be taking the columns of `genres`, `keywords`, `overview`, and `team` and then processing the text (cleaning the text)


`genres` & `keywords`

In [28]:
print(type(overall_df['genres'][0]))
print(overall_df['genres'][0])

<class 'list'>
['Action', 'Adventure', 'Fantasy', 'Science Fiction']


In [29]:
def clean_list_text(list):
    list = [words.lower() for words in list]
    list = [words.replace(' ', '') for words in list] # the reason for doing this... fiction and science fiction ain't the same thing!!!
    return list

In [30]:
overall_df['genres'] = overall_df['genres'].apply(clean_list_text)

In [31]:
overall_df['keywords'] = overall_df['keywords'].apply(clean_list_text)

In [32]:
overall_df

,genres,keywords,original_language,overview,popularity,runtime,title,movie_id,cast,director
0,"[action, adventure, fantasy, sciencefiction]","[cultureclash, future, spacewar, spacecolony, ...",en,"In the 22nd century, a paraplegic Marine is di...",150.437577,162.0,Avatar,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron
1,"[adventure, fantasy, action]","[ocean, drugabuse, exoticisland, eastindiatrad...",en,"Captain Barbossa, long believed to be dead, ha...",139.082615,169.0,Pirates of the Caribbean: At World's End,285,"[Johnny Depp, Orlando Bloom, Keira Knightley]",Gore Verbinski
2,"[action, adventure, crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...",en,A cryptic message from Bond’s past sends him o...,107.376788,148.0,Spectre,206647,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",Sam Mendes
3,"[action, crime, drama, thriller]","[dccomics, crimefighter, terrorist, secretiden...",en,Following the death of District Attorney Harve...,112.312950,165.0,The Dark Knight Rises,49026,"[Christian Bale, Michael Caine, Gary Oldman]",Christopher Nolan
4,"[action, adventure, sciencefiction]","[basedonnovel, mars, medallion, spacetravel, p...",en,"John Carter is a war-weary, former military ca...",43.926995,132.0,John Carter,49529,"[Taylor Kitsch, Lynn Collins, Samantha Morton]",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...
4804,"[action, crime, thriller]","[unitedstates–mexicobarrier, legs, arms, paper...",es,El Mariachi just wants to play his guitar and ...,14.269792,81.0,El Mariachi,9367,"[Carlos Gallardo, Jaime de Hoyos, Peter Marqua...",Robert Rodriguez
4805,"[comedy, romance]",[],en,A newlywed couple's honeymoon is upended by th...,0.642552,85.0,Newlyweds,72766,"[Edward Burns, Kerry Bishé, Marsha Dietlein]",Edward Burns
4806,"[comedy, drama, romance, tvmovie]","[date, loveatfirstsight, narration, investigat...",en,"""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,120.0,"Signed, Sealed, Delivered",231617,"[Eric Mabius, Kristin Booth, Crystal Lowe]",Scott Smith
4807,[],[],en,When ambitious New York attorney Sam is sent t...,0.857008,98.0,Shanghai Calling,126186,"[Daniel Henney, Eliza Coupe, Bill Paxton]",Daniel Hsia


#### Stemming the words and removing stopwords

In [33]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def more_clean(list):
    list = [words for words in list if not words in stop_words]
    list = [stemmer.stem(words) for words in list]
    return list

In [34]:
overall_df['keywords'] = overall_df['keywords'].apply(more_clean)

In [35]:
overall_df['genres'] = overall_df['genres'].apply(more_clean)

In [36]:
overall_df

,genres,keywords,original_language,overview,popularity,runtime,title,movie_id,cast,director
0,"[action, adventur, fantasi, sciencefict]","[cultureclash, futur, spacewar, spacecoloni, s...",en,"In the 22nd century, a paraplegic Marine is di...",150.437577,162.0,Avatar,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron
1,"[adventur, fantasi, action]","[ocean, drugabus, exoticisland, eastindiatradi...",en,"Captain Barbossa, long believed to be dead, ha...",139.082615,169.0,Pirates of the Caribbean: At World's End,285,"[Johnny Depp, Orlando Bloom, Keira Knightley]",Gore Verbinski
2,"[action, adventur, crime]","[spi, basedonnovel, secretag, sequel, mi6, bri...",en,A cryptic message from Bond’s past sends him o...,107.376788,148.0,Spectre,206647,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",Sam Mendes
3,"[action, crime, drama, thriller]","[dccomic, crimefight, terrorist, secretident, ...",en,Following the death of District Attorney Harve...,112.312950,165.0,The Dark Knight Rises,49026,"[Christian Bale, Michael Caine, Gary Oldman]",Christopher Nolan
4,"[action, adventur, sciencefict]","[basedonnovel, mar, medallion, spacetravel, pr...",en,"John Carter is a war-weary, former military ca...",43.926995,132.0,John Carter,49529,"[Taylor Kitsch, Lynn Collins, Samantha Morton]",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...
4804,"[action, crime, thriller]","[unitedstates–mexicobarri, leg, arm, paperknif...",es,El Mariachi just wants to play his guitar and ...,14.269792,81.0,El Mariachi,9367,"[Carlos Gallardo, Jaime de Hoyos, Peter Marqua...",Robert Rodriguez
4805,"[comedi, romanc]",[],en,A newlywed couple's honeymoon is upended by th...,0.642552,85.0,Newlyweds,72766,"[Edward Burns, Kerry Bishé, Marsha Dietlein]",Edward Burns
4806,"[comedi, drama, romanc, tvmovi]","[date, loveatfirstsight, narrat, investig, tea...",en,"""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,120.0,"Signed, Sealed, Delivered",231617,"[Eric Mabius, Kristin Booth, Crystal Lowe]",Scott Smith
4807,[],[],en,When ambitious New York attorney Sam is sent t...,0.857008,98.0,Shanghai Calling,126186,"[Daniel Henney, Eliza Coupe, Bill Paxton]",Daniel Hsia


`overview`
* First we'll have to break down these complete sentences into tokens, for this we'll use nltk's word tokenizer

In [37]:
from nltk.tokenize import word_tokenize
tokens = word_tokenize(overall_df['overview'][0])
x = [words for words in tokens if words.isalpha()]
print(x)

['In', 'the', 'century', 'a', 'paraplegic', 'Marine', 'is', 'dispatched', 'to', 'the', 'moon', 'Pandora', 'on', 'a', 'unique', 'mission', 'but', 'becomes', 'torn', 'between', 'following', 'orders', 'and', 'protecting', 'an', 'alien', 'civilization']


In [38]:
def overview_clean(text):
    tokens = word_tokenize(str(text))
    tokens = [words.lower() for words in tokens]
    tokens = [words for words in tokens if words.isalpha()]
    tokens = more_clean(tokens)
    return tokens

In [39]:
overall_df['overview'] = overall_df['overview'].apply(overview_clean)

In [40]:
overall_df

,genres,keywords,original_language,overview,popularity,runtime,title,movie_id,cast,director
0,"[action, adventur, fantasi, sciencefict]","[cultureclash, futur, spacewar, spacecoloni, s...",en,"[centuri, parapleg, marin, dispatch, moon, pan...",150.437577,162.0,Avatar,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron
1,"[adventur, fantasi, action]","[ocean, drugabus, exoticisland, eastindiatradi...",en,"[captain, barbossa, long, believ, dead, come, ...",139.082615,169.0,Pirates of the Caribbean: At World's End,285,"[Johnny Depp, Orlando Bloom, Keira Knightley]",Gore Verbinski
2,"[action, adventur, crime]","[spi, basedonnovel, secretag, sequel, mi6, bri...",en,"[cryptic, messag, bond, past, send, trail, unc...",107.376788,148.0,Spectre,206647,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",Sam Mendes
3,"[action, crime, drama, thriller]","[dccomic, crimefight, terrorist, secretident, ...",en,"[follow, death, district, attorney, harvey, de...",112.312950,165.0,The Dark Knight Rises,49026,"[Christian Bale, Michael Caine, Gary Oldman]",Christopher Nolan
4,"[action, adventur, sciencefict]","[basedonnovel, mar, medallion, spacetravel, pr...",en,"[john, carter, former, militari, captain, inex...",43.926995,132.0,John Carter,49529,"[Taylor Kitsch, Lynn Collins, Samantha Morton]",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...
4804,"[action, crime, thriller]","[unitedstates–mexicobarri, leg, arm, paperknif...",es,"[el, mariachi, want, play, guitar, carri, fami...",14.269792,81.0,El Mariachi,9367,"[Carlos Gallardo, Jaime de Hoyos, Peter Marqua...",Robert Rodriguez
4805,"[comedi, romanc]",[],en,"[newlyw, coupl, honeymoon, upend, arriv, respe...",0.642552,85.0,Newlyweds,72766,"[Edward Burns, Kerry Bishé, Marsha Dietlein]",Edward Burns
4806,"[comedi, drama, romanc, tvmovi]","[date, loveatfirstsight, narrat, investig, tea...",en,"[sign, seal, deliv, introduc, dedic, quartet, ...",1.444476,120.0,"Signed, Sealed, Delivered",231617,"[Eric Mabius, Kristin Booth, Crystal Lowe]",Scott Smith
4807,[],[],en,"[ambiti, new, york, attorney, sam, sent, shang...",0.857008,98.0,Shanghai Calling,126186,"[Daniel Henney, Eliza Coupe, Bill Paxton]",Daniel Hsia


`cast`
* Not a lot to do over here, we'll just remove the whitespace between names and surnames because Tom Hanks and Tom Hardy are not the same !!

In [41]:
def clean_ws(list):
    list = [words.replace(" ", "") for words in list]
    return list

In [42]:
overall_df['cast'] = overall_df['cast'].apply(clean_ws)

`director`
* We'll make a single element list of the names, this will make our work easier later on

In [43]:
def clean_dir(text):
    list = []
    if text is not None:
        list.append(text)
        list = [words.replace(" ", "") for words in list]
    return list

In [44]:
overall_df['director'] = overall_df['director'].apply(clean_dir)
overall_df['original_language'] = overall_df['original_language'].apply(clean_dir)

In [45]:
overall_df

,genres,keywords,original_language,overview,popularity,runtime,title,movie_id,cast,director
0,"[action, adventur, fantasi, sciencefict]","[cultureclash, futur, spacewar, spacecoloni, s...",[en],"[centuri, parapleg, marin, dispatch, moon, pan...",150.437577,162.0,Avatar,19995,"[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,"[adventur, fantasi, action]","[ocean, drugabus, exoticisland, eastindiatradi...",[en],"[captain, barbossa, long, believ, dead, come, ...",139.082615,169.0,Pirates of the Caribbean: At World's End,285,"[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,"[action, adventur, crime]","[spi, basedonnovel, secretag, sequel, mi6, bri...",[en],"[cryptic, messag, bond, past, send, trail, unc...",107.376788,148.0,Spectre,206647,"[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,"[action, crime, drama, thriller]","[dccomic, crimefight, terrorist, secretident, ...",[en],"[follow, death, district, attorney, harvey, de...",112.312950,165.0,The Dark Knight Rises,49026,"[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,"[action, adventur, sciencefict]","[basedonnovel, mar, medallion, spacetravel, pr...",[en],"[john, carter, former, militari, captain, inex...",43.926995,132.0,John Carter,49529,"[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]
...,...,...,...,...,...,...,...,...,...,...
4804,"[action, crime, thriller]","[unitedstates–mexicobarri, leg, arm, paperknif...",[es],"[el, mariachi, want, play, guitar, carri, fami...",14.269792,81.0,El Mariachi,9367,"[CarlosGallardo, JaimedeHoyos, PeterMarquardt]",[RobertRodriguez]
4805,"[comedi, romanc]",[],[en],"[newlyw, coupl, honeymoon, upend, arriv, respe...",0.642552,85.0,Newlyweds,72766,"[EdwardBurns, KerryBishé, MarshaDietlein]",[EdwardBurns]
4806,"[comedi, drama, romanc, tvmovi]","[date, loveatfirstsight, narrat, investig, tea...",[en],"[sign, seal, deliv, introduc, dedic, quartet, ...",1.444476,120.0,"Signed, Sealed, Delivered",231617,"[EricMabius, KristinBooth, CrystalLowe]",[ScottSmith]
4807,[],[],[en],"[ambiti, new, york, attorney, sam, sent, shang...",0.857008,98.0,Shanghai Calling,126186,"[DanielHenney, ElizaCoupe, BillPaxton]",[DanielHsia]


### Final Function

In [46]:
def to_string(list):
    return ' '.join(list)

`get_film_vector` : Helper function which will return the word embeddings for the tags of a particular movie when using Word2Vec model

In [47]:
def get_film_vector(idx, ser, model):
    vector = np.zeros(100)
    word_list = ser.iloc[idx].split()
    for words in word_list:
        vector += model.wv.get_vector(words)

    return vector

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from gensim.models.word2vec import Word2Vec
from scipy import spatial

def get_recommendations(title, weights = [1, 1, 1, 1, 1, 1], model = 1):
    
    # Creating the tags column using the weights, this can vary the recommendations greatly
    # Order of weights : Genres, Keywords, Overview, Cast, Director, Language
    
    Ser = weights[0]*overall_df['genres'] + weights[1]*overall_df['keywords'] + weights[2]*overall_df['overview'] + weights[3]*overall_df['cast'] + weights[4]*overall_df['director'] + weights[5]*overall_df['original_language']
    Ser = Ser.apply(to_string)
    
    indices = pd.Series(overall_df.index, index=overall_df['title']).drop_duplicates()
    
    if(model == 1):
        
        
        tfidf = TfidfVectorizer()
        tfidf_matrix = tfidf.fit_transform(Ser)
        cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
        
        idx = indices[title]

        sim_scores = enumerate(cosine_sim[idx])
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:11]
        movie_indices = [i[0] for i in sim_scores]

        for vals in overall_df['title'].iloc[movie_indices].values:
            print(vals)
            
    elif(model == 2):
        
        data = []
        for summs in Ser.values:
            list = summs.split()
            data.append(list)
            
        vec_model = Word2Vec(data, window=10, min_count=1, workers=4, sg = 1)
        
        idx = indices[title]
        the_vec = get_film_vector(idx, Ser, vec_model)
        sims = []
        for i in range(4809):
            vec2 = get_film_vector(i, Ser, vec_model)
            cossim = 1 - spatial.distance.cosine(the_vec, vec2)
            sims.append((cossim, i))

        sims = sorted(sims)
        recclist = []
        for cosi, ind in sims[-11:-1]:
            recclist.append(overall_df['title'].values[ind])

        recclist.reverse()

        for rex in recclist:
            print(rex)
        
        

### Results
#### **TFIDF**

In [49]:
get_recommendations('The Dark Knight Rises', [1, 1, 1, 1, 1, 1], 1) #Equal Weights given to all columns

The Dark Knight
Batman Returns
Batman Begins
Batman Forever
Batman
Batman
Batman: The Dark Knight Returns, Part 2
Batman & Robin
Batman v Superman: Dawn of Justice
Slow Burn


In [50]:
get_recommendations('The Dark Knight Rises', [1, 1, 10, 1, 1, 1], 1) # More weight given to Overview

The Dark Knight
Batman Forever
Batman Returns
Batman
Batman
Batman: The Dark Knight Returns, Part 2
Slow Burn
Batman Begins
Batman v Superman: Dawn of Justice
Batman & Robin


In [51]:
get_recommendations('The Dark Knight Rises', [1, 1, 1, 10, 1, 1], 1) # More weight given to Cast

Batman Begins
The Prestige
The Dark Knight
JFK
Miss Congeniality
The Weather Man
The Book of Eli
Children of Men
Secondhand Lions
The Big Short


In [52]:
get_recommendations('The Dark Knight Rises', [1, 1, 1, 1, 10, 1], 1) # More weight given to Director

The Dark Knight
Batman Begins
The Prestige
Insomnia
Inception
Interstellar
Memento
Batman Returns
Batman Forever
Batman


#### **Word2Vec**

In [53]:
get_recommendations('The Dark Knight Rises', [1, 1, 1, 1, 1, 1], 2) # Equal weights to all parameters

The Dark Knight
The Bourne Ultimatum
Dredd
Die Hard: With a Vengeance
Die Hard 2
Daylight
Mission: Impossible III
Eraser
Assault on Precinct 13
Mission: Impossible II


In [54]:
get_recommendations('The Dark Knight Rises', [1, 1, 10, 1, 1, 1], 2) # More weight to Overview

The Dark Knight
Batman Returns
Batman
Batman
Batman Forever
Batman Begins
Batman v Superman: Dawn of Justice
Max Payne
Batman: The Dark Knight Returns, Part 2
Righteous Kill


In [55]:
get_recommendations('The Dark Knight Rises', [1, 1, 1, 10, 1, 1], 2) # More weight to Cast

Batman Begins
The Prestige
Last Orders
The Dark Knight
A Bridge Too Far
Little Voice
Dressed to Kill
The Quiet American
On Deadly Ground
Terminator Salvation


In [56]:
get_recommendations('The Dark Knight Rises', [1, 1, 1, 1, 10, 1], 2) # More weight to Director

The Dark Knight
Batman Begins
Memento
Inception
Interstellar
Insomnia
The Prestige
Sky Captain and the World of Tomorrow
Hobo with a Shotgun
WarGames
